**1. Libraries**

In [1]:
from requests_html import HTMLSession
import pandas as pd
import re, os, urllib, shutil
import pandas as pd
import sqlite3

**2. Directories**

In [2]:
## Creating directories
path_raw = os.path.join(os.getcwd(), 'Projections')
path_national = os.path.join(path_raw, 'National')
path_departmental = os.path.join(path_raw, 'Departmental')
path_municipal = os.path.join(path_raw, 'Municipal')
if os.path.exists(path_raw) == False: 
    os.mkdir(path_raw)
if os.path.exists(path_national) == False: 
    os.mkdir(path_national)
if os.path.exists(path_departmental) == False: 
    os.mkdir(path_departmental)
if os.path.exists(path_municipal) == False: 
    os.mkdir(path_municipal)

**3. Connecting**

In [3]:
session = HTMLSession()

In [4]:
url = 'https://www.dane.gov.co/index.php/estadisticas-por-tema/demografia-y-poblacion/proyecciones-de-poblacion'

In [5]:
response = session.get(url)

**3.1 Scrapping**

In [6]:
links = response.html.absolute_links

In [7]:
National = list()
Departmental = list()
Municipal = list()
for link in links:
    if re.search('anexo-(area-sexo-edad-)*proyecciones-poblacion-Nacional[0-9]{4}_[0-9]{4}.xlsx$', link):
        National.append(link)
        continue
    if re.search('anexo-(area-sexo-edad-)*proyecciones-poblacion-departamental_[0-9]{4}-[0-9]{4}.xlsx$', link):
        Departmental.append(link)
        continue
    if re.search('anexo-(area-sexo-edad-)*proyecciones-poblacion-Municipal_[0-9]{4}-[0-9]{4}.xlsx$', link):
        Municipal.append(link)
        continue

**3.2 Download and export**

In [10]:
def get_data(links, pattern, path_level,csv_name, level):
    
    if level == 'Municipio':
        col_changes = {'DP':'Departamento', 'ÁREA GEOGRÁFICA':'Area', 'DPMP':'Municipio', 'AÑO':'Year'}
        column_names = ['Sex', 'Area']
        col_rem = ['Sexo_edad','DPNOM','MPIO']
    else:
        col_changes = {'DP':'Departamento', 'ÁREA GEOGRÁFICA':'Area', 'AÑO':'Year'}
        column_names = ['Sex', 'Area']
        col_rem = ['Sexo_edad','DPNOM']
        insert_projection = '''INSERT INTO Projections(departamento_id, area_id, sex_id, year, age, No_people) VALUES'''

    
    for web in links:
        name = re.findall(pattern, web)[0]
        path_doc = os.path.join(path_level, name)
        print('File:', name)
        if os.path.isfile(path_doc) == False:
            print('         Downloading------------')
            urllib.request.urlretrieve(web, path_doc)
        print('         Reading------------')
        df_all = pd.read_excel(path_doc, skiprows=11, dtype= 'str')

        cols_to_split = df_all.loc[:,'Hombres_0':].columns.values
        ids = df_all.loc[:,:'ÁREA GEOGRÁFICA'].columns.values
        print('         Converting Sex_age columns to rows------------')
        new_all = pd.melt(df_all, id_vars =ids, value_vars =cols_to_split, var_name ='Sexo_edad', value_name ='No_personas')

        print('         Spliting Sex_age column------------')
        new_all[["Sex", "Age"]] = new_all.Sexo_edad.str.split(pat='_', n = 1, expand = True)
        new_all = new_all.drop(columns=col_rem)
        
        new_all.rename(columns = col_changes, inplace = True)
        print('         Data frame cleaned------------')
        print(new_all.head(2))
        # Connecting with the database
        con = sqlite3.connect('Colombia_Demographic_data.sqlite', timeout=30)
        cur = con.cursor()
        
        z = 1
        if z == 1:
            ## area's table
            insert_area = 'INSERT INTO Area(name) VALUES'
            insert_area += '('
            
            for area in new_all['Area'].unique():
                area = area.capitalize()
                try:
                    select_query = 'SELECT id FROM '
                    select_query += 'Area' + ' WHERE name = ? ;'
                    cur.execute(select_query, (area, ))
                    cur.fetchone()[0]
                    continue
                except:
                    insert_area += '"'+str(area) + '"'+'),('
                    
            if insert_area != 'INSERT INTO Area(name) VALUES(':
                insert_area = insert_area[:-2] +';'
                cur.execute(insert_area)
            
            print('area inserted')
            
            ## Sex's table
            insert_sex = 'INSERT INTO Sex(name) VALUES'
            insert_sex += '('
            for sex in new_all['Sex'].unique():
                sex = sex.capitalize()
                try:
                    select_query = 'SELECT id FROM '
                    select_query += 'Sex' + ' WHERE name = ? ;'
                    cur.execute(select_query, (sex, ))
                    cur.fetchone()[0]
                    continue
                except:
                    insert_sex += '"'+str(sex) + '"'+'),('
            
            if insert_sex != 'INSERT INTO Sex(name) VALUES(':
                insert_sex = insert_sex[:-2] +';'
                cur.execute(insert_sex)
            con.commit()
            print('area and sex inserted')
            z += 1
            
        if level == 'Municipio':    
            insert_projection = '''INSERT INTO Projections(departamento_id, municipio_id, year, area_id, No_people, sex_id, age) VALUES'''
        else:
            insert_projection = '''INSERT INTO Projections(departamento_id, year , area_id, No_people, sex_id, age) VALUES'''

        r = 0
        for row in range(0,len(new_all)):
            i = 0
            insert_projection += '('
            for feature in new_all.iloc[row]:
                
                title   = new_all.columns.values[i]

                if title in column_names:
                    feature = feature.capitalize()
                    select_query = 'SELECT id FROM '
                    select_query += str(title) + ' WHERE name = ? ;'
                    cur.execute(select_query, (feature, ))
                    try:
                        feature_id = cur.fetchone()[0]
                        insert_projection += '"'+str(feature_id) + '"'+','
                    except:
                        print(title)
                        print(feature)
                        insert_projection += 'NULL,'
                elif pd.isna(feature):
                    insert_projection += 'NULL,'

                else:
                    insert_projection += '"' + str(feature) + '"'+','

                i +=1
            r +=1
            insert_projection = insert_projection[:-1] +'),'
            
            if r >= 10000:
                insert_projection = insert_projection[:-2] +');'
                cur.execute(insert_projection)
                r = 0
                if level == 'Municipio':    
                    insert_projection = '''INSERT INTO Projections(departamento_id, municipio_id, year, area_id, No_people, sex_id, age) VALUES'''
                else:
                    insert_projection = '''INSERT INTO Projections(departamento_id, year , area_id, No_people, sex_id, age) VALUES'''
            
        if r >0:
            insert_projection = insert_projection[:-2] +');'
            cur.execute(insert_projection)
            print('executed')
            con.commit()
            print('commited')
        con.close() 

In [11]:
get_data(National, "Nacional[0-9]{4}_[0-9]{4}.xlsx$", path_national, 'Projections\\National.csv', 'x')

File: Nacional1985_1992.xlsx
         Reading------------
         Converting Sex_age columns to rows------------
         Spliting Sex_age column------------
         Data frame cleaned------------
  Departamento  Year                               Area No_personas      Sex  \
0           00  1985                           Cabecera      240721  Hombres   
1           00  1985  Centros Poblados y Rural Disperso      162694  Hombres   

  Age  
0   0  
1   0  
area inserted
area and sex inserted
executed
commited
File: Nacional1950_1984.xlsx
         Reading------------
         Converting Sex_age columns to rows------------
         Spliting Sex_age column------------
         Data frame cleaned------------
  Departamento  Year                               Area No_personas      Sex  \
0           00  1950                           Cabecera       78665  Hombres   
1           00  1950  Centros Poblados y Rural Disperso      184737  Hombres   

  Age  
0   0  
1   0  
area inserted
area

In [12]:
get_data(Departmental, "departamental_[0-9]{4}-[0-9]{4}.xlsx$", path_departmental, 'Projections\\Departmental.csv', 'd')

File: departamental_1985-1992.xlsx
         Reading------------
         Converting Sex_age columns to rows------------
         Spliting Sex_age column------------
         Data frame cleaned------------
  Departamento  Year                               Area No_personas      Sex  \
0           05  1985                 Cabecera Municipal       26387  Hombres   
1           05  1985  Centros Poblados y Rural Disperso       22738  Hombres   

  Age  
0   0  
1   0  
area inserted
area and sex inserted
executed
commited
File: departamental_2018-2050.xlsx
         Reading------------
         Converting Sex_age columns to rows------------
         Spliting Sex_age column------------
         Data frame cleaned------------
  Departamento  Year                               Area No_personas      Sex  \
0           05  2018                 Cabecera Municipal       31398  Hombres   
1           05  2018  Centros Poblados y Rural Disperso       14488  Hombres   

  Age  
0   0  
1   0  
area i

In [13]:
get_data(Municipal, "Municipal_[0-9]{4}-[0-9]{4}.xlsx$", path_municipal,'Projections\\Municipal.csv', 'Municipio')

File: Municipal_2005-2017.xlsx
         Reading------------
         Converting Sex_age columns to rows------------
         Spliting Sex_age column------------
         Data frame cleaned------------
  Departamento Municipio  Year                               Area No_personas  \
0           05     05001  2005                 Cabecera Municipal       13812   
1           05     05001  2005  Centros Poblados y Rural Disperso         489   

       Sex Age  
0  Hombres   0  
1  Hombres   0  
area inserted
area and sex inserted
executed
commited
File: Municipal_1995-2004.xlsx
         Reading------------
         Converting Sex_age columns to rows------------
         Spliting Sex_age column------------
         Data frame cleaned------------
  Departamento Municipio  Year                               Area No_personas  \
0           05     05001  1995                 Cabecera Municipal       16228   
1           05     05001  1995  Centros Poblados y Rural Disperso         681   

     

**4. Remove directories**

In [ ]:
shutil.rmtree(path_national)
shutil.rmtree(path_departmental)
shutil.rmtree(path_municipal)